This notebook briefly walks through fitting the four linear Bayesian regression calibrations for global core top $\delta^{18}\mathrm{O}_{\mathrm{c}}$ and SSTs.

Notebook settings for graphics and load some libraries.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc as pm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lgmproxies.datasets.tierney import get_repo_path
prefix = get_repo_path("brews/d18oc_sst").as_posix() + "/notebooks/"
prefix

Set paths to our core top data. These are also given as supplemental information in the paper.

In [ ]:
coretop_path = prefix + '../data/parsed/coretops.csv'
coretop_grid_path = prefix + '../data/parsed/coretops_grid.csv'

# Read coretop data and parse

This should be straightforward.

In [ ]:
coretops_raw = pd.read_csv(coretop_path)
coretops_grid = pd.read_csv(coretop_grid_path)

In [ ]:
coretops_raw['foramtype'] = coretops_raw['species'].astype('category')
coretops_grid['foramtype'] = coretops_grid['species'].astype('category')

Take a quick look at the data to see what we're dealing with... (and make sure everything is there)

In [ ]:
coretops_raw['foramtype'].unique()

In [ ]:
coretops_raw['foramtype'].value_counts()

In [ ]:
coretops_grid.head()

# MCMC models

## Annual SST pooled model

This is the model setup that pools all foram species together and calibrates on annual-mean SSTs.

Models is:
\begin{align}
\delta^{18}O_c = \alpha + \beta * T + \delta^{18}O_{sw} - 0.27 + \epsilon \\
\end{align}

with parameters:
\begin{align}
\alpha \sim \mathcal{N}(3, 4) \\
\beta \sim \mathcal{N}(-0.2, 1) \\
\epsilon \sim \mathcal{N}(0, \tau) \\
\tau \sim \mathrm{HalfCauchy}(1)
\end{align}


Copy the original `coretops_grid` to `coretops`, which is our working copy. Then we make a `temp` column from the annual SST data.

In [ ]:
coretops = coretops_grid.copy()
coretops['temp'] = coretops['t_annual']

This gives you an idea what the data looks like:

In [ ]:
print(coretops.shape)
coretops.head()

This is the number of gridpoints we have for each foram species.

In [ ]:
coretops.foramtype.value_counts()

Now define the model and sample it.

In [ ]:
coretops['d18osw'].hist()

In [ ]:
temp = coretops['temp'].values
d18osw = coretops['d18osw'].values
d18oc = coretops['d18oc'].values

with pm.Model() as model:
    # Intercept and slope
    a = pm.Normal('a', mu=3.0, sigma=2)
    b = pm.Normal('b', mu=-0.2, sigma=1)

    # Model error
    tau = pm.HalfCauchy('tau', beta=1)

    # Likelihood
    d18oc_est = a + b * temp + (d18osw - 0.27)
    pm.Deterministic('d18oc_est', d18oc_est)
    likelihood_d18oc = pm.Normal('likelihood_d18oc', mu=d18oc_est, sigma=tau,
                                 observed=d18oc)
    trace = pm.sample(draws=5000, tune=1000, chains=2, init='jitter+adapt_diag', random_seed=123)

Very basic diagnostic plots showing the sampling and posterior distribution of the parameters. Remember that the trace plot (below) has two lines on each plot, one for each chain.

In [ ]:
import arviz
arviz.plot_trace(trace)

In [ ]:
arviz.plot_forest(trace)

In [ ]:
pm.summary(trace, hdi_prob=0.95)

Here is our PSIS-LOO statistics, for cross-validation.

In [ ]:
with model:
    pm.compute_log_likelihood(trace)

In [ ]:
pm.stats.loo(trace, model)

Save the model and trace for later use

In [ ]:
from pathlib import Path
import cloudpickle as cp
resultsdir = Path("../modelresults/")
resultsdir.mkdir(parents=True, exist_ok=True)
cp.dump(model, open(resultsdir/'deltao18_annual.cpkl', 'wb'))
trace.to_netcdf(resultsdir/'deltao18_annual.nc')

## Seasonal SST pooled model

Same deal as before. This is a model setup that pools all foram species together but calibrates against the seasonal SSTs instead of annual.

Models is:
\begin{align}
\delta^{18}O_c = \alpha + \beta * T + \delta^{18}O_{sw} - 0.27 + \epsilon \\
\end{align}

with parameters:
\begin{align}
\alpha \sim \mathcal{N}(3, 4) \\
\beta \sim \mathcal{N}(-0.2, 1) \\
\epsilon \sim \mathcal{N}(0, \tau) \\
\tau \sim \mathrm{HalfCauchy}(1)
\end{align}


In [ ]:
coretops = coretops_grid.copy()
coretops['temp'] = coretops['t_seasonal']

In [ ]:
print(coretops.shape)
coretops.head()

Now define the model and sample...

In [ ]:
temp = coretops['temp'].values
d18osw = coretops['d18osw'].values
d18oc = coretops['d18oc'].values

with pm.Model() as model:
    # Intercept and slope
    a = pm.Normal('a', mu=3.0, sd=2)
    b = pm.Normal('b', mu=-0.2, sd=1)

    # Model error
    tau = pm.HalfCauchy('tau', beta=1)

    # Likelihood
    d18oc_est = a + b * temp + (d18osw - 0.27)
    likelihood_d18oc = pm.Normal('likelihood_d18oc', mu=d18oc_est, sd=tau,
                                 observed=d18oc)
    trace = pm.sample(draws=5000, tune=1000, chains=2, init='jitter+adapt_diag', random_seed=123)

Diagnostic plots...

In [ ]:
pm.plots.traceplot(trace)

In [ ]:
pm.plots.forestplot(trace)

PSIS-LOO, the cross-validation statistic:

In [ ]:
pm.stats.loo(trace, model)

## Annual SST hierarchical model

This is the hierarchical model setup, calibrated on annual SSTs. We're allowing some wiggle room in the parameters, for the individual species.

Models is:
\begin{align}
\delta^{18}O_c = \alpha_i + \beta_i * T + \delta^{18}O_{sw} - 0.27 + \epsilon \\
\end{align}

using parameters set for individual foram species ($i$):
\begin{align}
\epsilon \sim \mathcal{N}(0, \tau_i) \\
\alpha_i \sim \mathcal{N}(\mu_\alpha, \sigma_\alpha) \\
\beta_i \sim \mathcal{N}(\mu_\beta, \sigma_\beta) \\
\tau_i \sim \mathrm{\Gamma}( \frac{\sigma_m^2} {\sigma_d^2}, \frac{\sigma_m} {\sigma_d^2} ) \\
\end{align}

and hyperparameters:
\begin{align}
\mu_\alpha \sim \mathcal{N}(3, 4) \\
\mu_\beta \sim \mathcal{N}(-0.2, 1) \\
\sigma_\alpha \sim \mathrm{HalfCauchy}(0.5) \\
\sigma_\beta \sim \mathrm{HalfCauchy}(0.25) \\
\sigma_m \sim \mathrm{HalfCauchy}(1) \\
\sigma_d \sim \mathrm{HalfCauchy}(1) \\
\end{align}



The rest follows what we had before.

Let's make a new copy of the core top data and set `temp`:

In [ ]:
coretops = coretops_grid.copy()
coretops['temp'] = coretops['t_annual']

Define the model and sample from it...

In [ ]:
temp = coretops['temp'].values
d18osw = coretops['d18osw'].values
d18oc = coretops['d18oc'].values
foramtype = coretops['foramtype'].cat.codes
n_foram = len(set(foramtype))


with pm.Model() as model:
    # Hyperparameters
    mu_a = pm.Normal('mu_a', mu=3, sigma=2)
    sigma_a = pm.HalfCauchy('sigma_a', beta=0.5)

    mu_b = pm.Normal('mu_b', mu=-0.2, sigma=1)
    sigma_b = pm.HalfCauchy('sigma_b', beta=0.25)

    sigma_m = pm.HalfCauchy('sigma_m', beta=1)
    sigma_d = pm.HalfCauchy('sigma_d', beta=1)

    # Intercept and slope
    a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=n_foram)
    b = pm.Normal('b', mu=mu_b, sigma=sigma_b, shape=n_foram)

    # Model error
    tau = pm.Gamma('tau', alpha=sigma_m**2 / sigma_d**2,
                          beta=sigma_m / sigma_d**2,
                          shape=n_foram)

    # Likelihood
    d18oc_est = a[foramtype] + b[foramtype] * temp + (d18osw - 0.27)
    pm.Deterministic('d18oc_est', d18oc_est)
    likelihood_d18oc = pm.Normal('likelihood_d18oc', mu=d18oc_est,
                                 sigma=tau[foramtype], observed=d18oc)
    trace = pm.sample(draws=5000, tune=1000, chains=2, init='jitter+adapt_diag', random_seed=123)

In [ ]:
from pathlib import Path
import cloudpickle as cp
resultsdir = Path("../modelresults/")
resultsdir.mkdir(parents=True, exist_ok=True)
cp.dump(model, open(resultsdir/'deltao18_hierarchical_annual.cpkl', 'wb'))
trace.to_netcdf(resultsdir/'deltao18_hierarchical_annual.nc')

In [ ]:
from lgmproxies.datasets.tierney import DeltaO18, DeloSWMalevitch, DeltaO18Hierarchical

def convert_d18o_df_tierney(df, lgm=False, hierarchical=False):
    deloswvals = delosw.interpolate(df["Longitude"], df["Latitude"])
    if hierarchical:
        return deltaO18Hierarchicalmodel.to_sst(df["ProxyValue"].values, df["Species"].values, deloswvals, seed=3422)
    else:
        return deltaO18model.to_sst(df["ProxyValue"].values, deloswvals, seed=345)

delosw = DeloSWMalevitch()
deltaO18model = DeltaO18.load("../modelresults/deltao18_annual.cpkl", "../modelresults/deltao18_annual.nc")
deltaO18Hierarchicalmodel = DeltaO18Hierarchical.load("../modelresults/deltao18_hierarchical_annual.cpkl", "../modelresults/deltao18_hierarchical_annual.nc")

In [ ]:
trace = deltaO18Hierarchicalmodel.trace
model = deltaO18Hierarchicalmodel.model

In [ ]:
temp = coretops['temp'].values
d18osw = coretops['d18osw'].values
d18oc = coretops['d18oc'].values
foramtype = coretops['foramtype'].cat.codes

In [ ]:
pooled = np.median(deltaO18model.to_sst(d18oc, d18osw, seed=123), axis=0)
hierarchical = np.median(deltaO18Hierarchicalmodel.to_sst(d18oc, foramtype, d18osw, seed=123), axis=0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 6))
r2pooled = np.corrcoef(pooled, temp)[0, 1]**2
rms = np.sqrt(np.mean((pooled - temp)**2))
r2h = np.corrcoef(hierarchical, temp)[0, 1]**2
rmsh = np.sqrt(np.mean((hierarchical - temp)**2))
ax.scatter(temp, pooled, label=f'Pooled\n$r^2=${r2pooled:.2f}\nrms={rms:.1f}C', s=3)
ax.scatter(temp, hierarchical, label=f'Hierarchical\n$r^2=${r2h:.2f}\nrms={rmsh:.1f}C', s=3)
ax.set_xlabel('Temperature (C)')
ax.set_ylabel('Estimated SST (C)')
ax.set_title('Pooled vs Hierarchical SST Estimates')
ax.set_aspect('equal')
ax.grid(True)
ax.legend()
f.savefig('../images/d18o_sst_pooled_vs_hierarchical.png', dpi=300, bbox_inches='tight')

In [ ]:
import arviz as az
f, ax = plt.subplots(1, 1, figsize=(10, 6))
pooled = az.extract(deltaO18model.trace.posterior['d18oc_est'])['d18oc_est'].median(dim="sample").values
hierarchical = az.extract(deltaO18Hierarchicalmodel.trace.posterior['d18oc_est'])['d18oc_est'].median(dim="sample").values
r2pooled = np.corrcoef(pooled, d18oc)[0, 1]**2
rms = np.sqrt(np.mean((pooled - d18oc)**2))
r2h = np.corrcoef(hierarchical, d18oc)[0, 1]**2
rmsh = np.sqrt(np.mean((hierarchical - d18oc)**2))
ax.scatter(d18oc, pooled, label=f'Pooled\n$r^2=${r2pooled:.2f}\nrms={rms:.2f}%$_o$', s=3)
ax.scatter(d18oc, hierarchical, label=f'Hierarchical\n$r^2=${r2h:.2f}\nrms={rmsh:.2f}%$_o$', s=3)
ax.set_xlabel('Observed d18O (per mil)')
ax.set_ylabel('Estimated d18O (per mil)')
ax.set_title('Pooled vs Hierarchical deltaO18 Estimates')
ax.set_aspect('equal')
ax.grid(True)
ax.legend()

f.savefig('../images/d18o_pooled_vs_hierarchical.png', dpi=300, bbox_inches='tight')

Diagnostic plots and such... same as before.

In [ ]:
dict(zip(coretops['foramtype'].cat.codes, coretops['foramtype'].values))

In [ ]:
import arviz as az
az.plot_trace(trace)

In [ ]:
az.plot_forest(trace)

...and the PSIS-LOO cross-validation statistic:

In [ ]:
with model:
    pm.compute_log_likelihood(trace)

In [ ]:
pm.stats.loo(trace, model)

## Seasonal SST hierarchical model

Here we use the same hierarchical model setup, but calibrated on seasonal SSTs.

Models is:
\begin{align}
\delta^{18}O_c = \alpha_i + \beta_i * T + \delta^{18}O_{sw} - 0.27 + \epsilon \\
\end{align}

using parameters set for individual foram species ($i$):
\begin{align}
\epsilon \sim \mathcal{N}(0, \tau_i) \\
\alpha_i \sim \mathcal{N}(\mu_\alpha, \sigma_\alpha) \\
\beta_i \sim \mathcal{N}(\mu_\beta, \sigma_\beta) \\
\tau_i \sim \mathrm{\Gamma}( \frac{\sigma_m^2} {\sigma_d^2}, \frac{\sigma_m} {\sigma_d^2} ) \\
\end{align}

and hyperparameters:
\begin{align}
\mu_\alpha \sim \mathcal{N}(3, 4) \\
\mu_\beta \sim \mathcal{N}(-0.2, 1) \\
\sigma_\alpha \sim \mathrm{HalfCauchy}(0.5) \\
\sigma_\beta \sim \mathrm{HalfCauchy}(0.25) \\
\sigma_m \sim \mathrm{HalfCauchy}(1) \\
\sigma_d \sim \mathrm{HalfCauchy}(1) \\
\end{align}



In [ ]:
coretops = coretops_grid.copy()
coretops['temp'] = coretops['t_seasonal']

Fit the model and sample it...

In [ ]:
temp = coretops['temp'].values
d18osw = coretops['d18osw'].values
d18oc = coretops['d18oc'].values
foramtype = coretops['foramtype'].cat.codes
n_foram = len(set(foramtype))


with pm.Model() as model:
    # Hyperparameters
    mu_a = pm.Normal('mu_a', mu=3, sd=2)
    sigma_a = pm.HalfCauchy('sigma_a', beta=0.5)

    mu_b = pm.Normal('mu_b', mu=-0.2, sd=1)
    sigma_b = pm.HalfCauchy('sigma_b', beta=0.25)

    sigma_m = pm.HalfCauchy('sigma_m', beta=1)
    sigma_d = pm.HalfCauchy('sigma_d', beta=1)

    # Intercept and slope
    a = pm.Normal('a', mu=mu_a, sd=sigma_a, shape=n_foram)
    b = pm.Normal('b', mu=mu_b, sd=sigma_b, shape=n_foram)

    # Model error
    tau = pm.Gamma('tau', alpha=sigma_m**2 / sigma_d**2,
                          beta=sigma_m / sigma_d**2,
                          shape=n_foram)

    # Likelihood
    d18oc_est = a[foramtype] + b[foramtype] * temp + (d18osw - 0.27)
    likelihood_d18oc = pm.Normal('likelihood_d18oc', mu=d18oc_est,
                                 sd=tau[foramtype], observed=d18oc)
    trace = pm.sample(draws=5000, tune=1000, chains=2, init='jitter+adapt_diag', random_seed=123)

Diagnostics...

In [ ]:
pm.plots.traceplot(trace)

In [ ]:
pm.plots.forestplot(trace)

And our PSIS-LOO cross-validation statistic...

In [ ]:
pm.stats.loo(trace, model)